In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from datetime import datetime 


def strip(text):
    try:
        return text.strip()
    except AttributeError:
        return text

def make_int(text):
    return int(text.strip('" '))

def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

In [2]:
'''
Data Sources
Application Responses (Simplified) -> download tab as a CSV
Volunteer Preferences -> download tab as a CSV
'''


responses = pd.read_csv('2024 Bonnaroo Volunteer Planning (previously Bonnaroo Volunteer Short List 2024) - Application Responses (Simplified).csv', 
                        converters = {'Last name' : strip,
                                    'First name' : strip})
preferences = pd.read_csv('2024 Bonnaroo Volunteer Planning (previously Bonnaroo Volunteer Short List 2024) - Volunteer Preferences.csv', header=1, 
                          converters = {'Unnamed: 0' : strip,
                                    'Unnamed: 1' : strip})

In [3]:
responses['fullName'] = responses['First name'] + responses['Last name']
preferences['fullName'] = preferences['Unnamed: 0'] + preferences['Unnamed: 1']

responses = fuzzy_merge(responses, preferences, 'fullName', 'fullName', threshold=70, limit=1)
filled_preferences = preferences.reset_index(drop=True)

In [4]:
#Filling Out Preferences Columns
for index, row in preferences.iterrows():
    single_response = responses.loc[responses['matches'] == row['fullName']]
    
    '''
    'Tear Down', 'Media', 'Workshop Support', 'Data Specialist',
       'Playground Coordinator', 'Set Up Lead', 'Tear Down Lead',
       'Preferred Time (Morning)', 'Preferred Time (Afternoon)',
       'Preferred Time (Evening)', 'Arrival', 'Departure', 'Preferred Partner',
       'Unnamed: 18',
       'last year's sheet for reference on formatting under Volunteer Preference tab. *Make preferred partners the same color*',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Mon 6/10', 'Load-in', '9a-6p (PR staff on site)',
    '''
    
    #Core Roles 
    if single_response['Which roles are you most interested in? (See role descriptions above)'].str.contains('Playground').any():
        filled_preferences.loc[filled_preferences['fullName'] == single_response['matches'].reset_index(drop=True)[0].strip(), 'Playground'] = 'x' 
    if single_response['Which roles are you most interested in? (See role descriptions above)'].str.contains('Activator').any():
        filled_preferences.loc[filled_preferences['fullName'] == single_response['matches'].reset_index(drop=True)[0].strip(), 'Activations'] = 'x'
    if single_response['Which roles are you most interested in? (See role descriptions above)'].str.contains('Set Up').any():
        filled_preferences.loc[filled_preferences['fullName'] == single_response['matches'].reset_index(drop=True)[0].strip(), 'Set Up'] = 'x'
    if single_response['Which roles are you most interested in? (See role descriptions above)'].str.contains('Tear Down').any():
        filled_preferences.loc[filled_preferences['fullName'] == single_response['matches'].reset_index(drop=True)[0].strip(), 'Tear Down'] = 'x'
        
        
    #Specialized Roles
    if single_response['Are you interested in any specialized roles?'].str.contains('Media Crew').any():
        filled_preferences.loc[filled_preferences['fullName'] == single_response['matches'].reset_index(drop=True)[0].strip(), 'Media'] = 'x'
    if single_response['Are you interested in any specialized roles?'].str.contains('Workshop Support').any():
        filled_preferences.loc[filled_preferences['fullName'] == single_response['matches'].reset_index(drop=True)[0].strip(), 'Workshop Support'] = 'x'
    if single_response['Are you interested in any specialized roles?'].str.contains('Playground Coordinator').any():
        filled_preferences.loc[filled_preferences['fullName'] == single_response['matches'].reset_index(drop=True)[0].strip(), 'Playground Coordinator'] = 'x'
    if single_response['Are you interested in any specialized roles?'].str.contains('Data Specialist').any():
        filled_preferences.loc[filled_preferences['fullName'] == single_response['matches'].reset_index(drop=True)[0].strip(), 'Data Specialist'] = 'x'
    if single_response['Are you interested in any specialized roles?'].str.contains('Set Up Lead').any():
        filled_preferences.loc[filled_preferences['fullName'] == single_response['matches'].reset_index(drop=True)[0].strip(), 'Set Up Lead'] = 'x'
    if single_response['Are you interested in any specialized roles?'].str.contains('Tear Down Lead').any():
        filled_preferences.loc[filled_preferences['fullName'] == single_response['matches'].reset_index(drop=True)[0].strip(), 'Tear Down Lead'] = 'x'
        
    #Preferred Times of Day
    if single_response['Preferred Time of Day to Work'].str.contains('Morning').any():
        filled_preferences.loc[filled_preferences['fullName'] == single_response['matches'].reset_index(drop=True)[0].strip(), 'Preferred Time (Morning)'] = 'x'
    if single_response['Preferred Time of Day to Work'].str.contains('Afternoon').any():
        filled_preferences.loc[filled_preferences['fullName'] == single_response['matches'].reset_index(drop=True)[0].strip(), 'Preferred Time (Afternoon)'] = 'x'
    if single_response['Preferred Time of Day to Work'].str.contains('Evening').any():
        filled_preferences.loc[filled_preferences['fullName'] == single_response['matches'].reset_index(drop=True)[0].strip(), 'Preferred Time (Evening)'] = 'x'
    if single_response['Preferred Time of Day to Work'].str.contains('No Preference').any():
        filled_preferences.loc[filled_preferences['fullName'] == single_response['matches'].reset_index(drop=True)[0].strip(), 'Preferred Time (Morning)'] = 'x'
        filled_preferences.loc[filled_preferences['fullName'] == single_response['matches'].reset_index(drop=True)[0].strip(), 'Preferred Time (Afternoon)'] = 'x'
        filled_preferences.loc[filled_preferences['fullName'] == single_response['matches'].reset_index(drop=True)[0].strip(), 'Preferred Time (Evening)'] = 'x'

        
    #Travel 
    if single_response['When are you able to arrive on site? '].str.contains('').any():
        filled_preferences.loc[filled_preferences['fullName'] == single_response['matches'].reset_index(drop=True)[0].strip(), 'Arrival'] = single_response['When are you able to arrive on site? '].reset_index(drop=True)[0].strip()
    if single_response['When do you plan to leave the festival?'].str.contains('').any():
        filled_preferences.loc[filled_preferences['fullName'] == single_response['matches'].reset_index(drop=True)[0].strip(), 'Departure'] = single_response['When do you plan to leave the festival?'].reset_index(drop=True)[0].strip()
    
    #Preferred Partner
    if single_response['Preferred Volunteer Partner'].str.contains(' ').any():
        filled_preferences.loc[filled_preferences['fullName'] == single_response['matches'].reset_index(drop=True)[0].strip(), 'Preferred Partner'] = single_response['Preferred Volunteer Partner'].reset_index(drop=True)[0].strip()

C:\Users\mcraf\AppData\Local\Temp\ipykernel_18316\1929535260.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'x' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  filled_preferences.loc[filled_preferences['fullName'] == single_response['matches'].reset_index(drop=True)[0].strip(), 'Set Up'] = 'x'
C:\Users\mcraf\AppData\Local\Temp\ipykernel_18316\1929535260.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'x' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  filled_preferences.loc[filled_preferences['fullName'] == single_response['matches'].reset_index(drop=True)[0].strip(), 'Playground Coordinator'] = 'x'
C:\Users\mcraf\AppData\Local\Temp\ipykernel_18316\1929535260.py:37: FutureWarning: Setting an item of incompatible dtype is deprecat

In [5]:
#Build Preferences based on response

#Core Roles 
if single_response['Which roles are you most interested in? (See role descriptions above)'].str.contains('Playground').any():
    print('1')
if single_response['Which roles are you most interested in? (See role descriptions above)'].str.contains('Activator').any():
    print('2')
if single_response['Which roles are you most interested in? (See role descriptions above)'].str.contains('Set Up').any():
    print('3')
if single_response['Which roles are you most interested in? (See role descriptions above)'].str.contains('Tear Down').any():
    print('4')
    
#Specialized Roles
if single_response['Are you interested in any specialized roles?'].str.contains('Media Crew').any():
    print('1')
if single_response['Are you interested in any specialized roles?'].str.contains('Workshop Support').any():
    print('1')
if single_response['Are you interested in any specialized roles?'].str.contains('Playground Coordinator').any():
    print('1')
if single_response['Are you interested in any specialized roles?'].str.contains('Data Specialist').any():
    print('1')
if single_response['Are you interested in any specialized roles?'].str.contains('Set Up Lead').any():
    print('1')
if single_response['Are you interested in any specialized roles?'].str.contains('Tear Down Lead').any():
    print('1')
    
#Preferred Times of Day
if single_response['Preferred Time of Day to Work'].str.contains('Morning').any():
    print('1')
if single_response['Preferred Time of Day to Work'].str.contains('Afternoon').any():
    print('2')
if single_response['Preferred Time of Day to Work'].str.contains('Evening').any():
    print('3')
if single_response['Preferred Time of Day to Work'].str.contains('No Preference').any():
    print('4')
    
#Travel 
if single_response['When are you able to arrive on site? '].str.contains(' ').any():
    print('1')
if single_response['When do you plan to leave the festival?'].str.contains(' ').any():
    print('1')

#Preferred Partner
if single_response['Preferred Volunteer Partner'].str.contains(' ').any():
    print(single_response['Preferred Volunteer Partner'].reset_index(drop=True)[0].strip())

3
4
1
3
1


In [6]:
filled_preferences.fillna('', inplace=True)

C:\Users\mcraf\AppData\Local\Temp\ipykernel_18316\3896983854.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  filled_preferences.fillna('', inplace=True)


In [7]:
filled_preferences[['Unnamed: 0', 'Unnamed: 1', 'Playground', 'Activations', 'Set Up',
       'Tear Down', 'Media', 'Workshop Support', 'Data Specialist',
       'Playground Coordinator', 'Set Up Lead', 'Tear Down Lead',
       'Preferred Time (Morning)', 'Preferred Time (Afternoon)',
       'Preferred Time (Evening)', 'Arrival', 'Departure', 'Preferred Partner']]

,Unnamed: 0,Unnamed: 1,Playground,Activations,Set Up,Tear Down,Media,Workshop Support,Data Specialist,Playground Coordinator,Set Up Lead,Tear Down Lead,Preferred Time (Morning),Preferred Time (Afternoon),Preferred Time (Evening),Arrival,Departure,Preferred Partner
0,Christian (Kip),Hunsinger,x,x,,,,x,x,,,,x,x,,Wednesday by 1,Monday morning/afternoon.,Noah Keim
1,Jess,Kessie,x,x,,x,x,x,,,,,x,x,,Yes indeed!,Monday night,Malia Rogers!!!
2,Malia,Rogers,,,,,,,,,,,,,,,,
3,Shanti,Cuizon-Burden,,,,,,,,,,,,,,,,
4,Zoe Rene,Mais,,,,,,,,,,,,,,,,
5,"Devin ""Polaroid""",Patrick,x,x,x,,x,x,x,x,x,,x,x,x,I can be on site by Tuesday evening,Monday Morning,I’m flexible
6,Darby,Holmes,x,x,x,x,,x,,,,,x,x,x,Anytime before or on Wednesday,When we’re done tearing down and y’all tell me...,MO BABY!!!
7,Maura (Mo),Poole,x,x,x,x,x,x,,x,,,x,x,,I have open availability in the summer. I woul...,Monday,Darby :)
8,Noah,Keim,x,x,,x,,x,x,x,,x,x,x,x,Wednesday afternoon,Monday afternoon,
9,Monica,Mendoza,x,x,,,,x,,x,,,x,x,,Wednesday morning.,Sunday night or early Monday- heading to Elect...,"Phil Suozzi, Lexton Deleon, and CJ Bradley!"


In [8]:
filled_preferences.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Playground', 'Activations', 'Set Up',
       'Tear Down', 'Media', 'Workshop Support', 'Data Specialist',
       'Playground Coordinator', 'Set Up Lead', 'Tear Down Lead',
       'Preferred Time (Morning)', 'Preferred Time (Afternoon)',
       'Preferred Time (Evening)', 'Arrival', 'Departure', 'Preferred Partner',
       'Unnamed: 18',
       'last year's sheet for reference on formatting under Volunteer Preference tab. *Make preferred partners the same color*',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Mon 6/10', 'Load-in', '9a-6p (PR staff on site)',
       'fullName'],
      dtype='object')

In [26]:
'''
Volunteer Preferences Management 
For each volunteer:
    Find intersection of their top artist and the schedule
    Write Name 1 into cell 
    Repeat for second and third top artists 
For each pair of preferred partners:
    color the pair's name the same
    
Datasource 1: Artist Worksheet tab with LookupName, Day, and Time
Datasource 2: Artist worksheet tab with Original and renamed to deal with messiness

Matching flow:
For each artist in row:
    look up the Renamed value from the Original value
    look up the renamed value in the time table
'''

"\nVolunteer Preferences Management \nFor each volunteer:\n    Find intersection of their top artist and the schedule\n    Write Name 1 into cell \n    Repeat for second and third top artists \nFor each pair of preferred partners:\n    color the pair's name the same\n    \nDatasource 1: Artist Worksheet tab with LookupName, Day, and Time\nDatasource 2: Artist worksheet tab with Original and renamed to deal with messiness\n\nMatching flow:\nFor each artist in row:\n    look up the Renamed value from the Original value\n    look up the renamed value in the time table\n"

In [27]:
artists = pd.read_csv("C:/Users/mcraf/Downloads/2024 Artist Worksheet.csv", usecols=[0,1 ])
artists = artists.drop_duplicates()

set_times = pd.read_csv("C:/Users/mcraf/Downloads/2024 Artist Worksheet.csv", usecols=['LookupName', 'Day', 'Time'])
set_times.dropna(inplace = True)
set_times.fillna('', inplace=True)


In [28]:
def get_time(x, y):
    key = x.split(" to ")[y]
    return key

def iferror(success, failure, *exceptions):
    try:
        return success()
    except exceptions or Exception:
        return failure

set_times['Start Time'] = set_times['Time'].apply(lambda x: get_time(x, 0))
set_times['End Time'] = set_times['Time'].apply(lambda x: get_time(x, 1))


In [29]:
columns = [
      'fullName'
    , 'matches'
    , 'First name'
    , 'Last name'
    , 'Preferred Volunteer Partner'
    , 'Top Artist #1'
    , 'Top Artist #2'
    , 'Top Artist #3'
]

step2_responses = responses[columns]
step2_responses_cleaned = step2_responses

In [30]:
artists = artists.drop_duplicates()

In [31]:
# rounding set times 
def round_time(x, y):
    new_x = x
    if y == 's':
        if x[2:] == '15':
            new_x = x[:2] + '00'
        elif x[2:] == '45':
            new_x = x[:2] + '30'
    elif y == 'e':
        if x[2:] == '15':
            new_x = x[:2] + '30'
        elif x[2:] == '45':
            new_x = str(int(x[:2])+1) + '00'
    else:
        new_x = x
        return new_x
    return new_x
'''
20240505 - I got lazy about the rounding around midnight since I didn't think we'd have to care, 
but if we're groove chuting we should look at incrementing the day if needed

I did spot check and I'm not sure this applied to any important artists though
'''

#rounding start times 
set_times['Start Time'] = set_times['Start Time'].apply(lambda x: round_time(x, 's'))

#rounding end times 
set_times['End Time'] = set_times['End Time'].apply(lambda x: round_time(x, 'e'))

In [32]:
set_times

,LookupName,Day,Time,Start Time,End Time
0,Ashnikko,Sunday,1815 to 1915,1800,1930
1,Brittany howard,Saturday,1730 to 1830,1730,1830
2,Cage the Elephant,Saturday,2030 to 2145,2030,2200
3,Chappell Roan,Sunday,1600 to 1700,1600,1700
4,Cigarettes After Sex,Saturday,2145 to 2245,2130,2300
5,Dashboard Confessional,Saturday,2030 to 2230,2030,2230
6,Diplo,Sunday,0100 to 0300,0100,0300
7,Dr Fresch,Friday,2100 to 2200,2100,2200
8,Fisher,Friday,0045 to 0200,0030,0200
9,Four Tet,Sunday,1700 to 1830,1700,1830


In [51]:
top_artist_cols = [
    'Volunteer Top Artists'
    , 'Unnamed: 20'
    , 'Unnamed: 21'
    , 'Unnamed: 22'
    , 'Unnamed: 23'
]
#preferences_matrix = filled_preferences[top_artist_cols]
preferences_matrix = pd.read_csv("C:/Users/mcraf/Downloads/Volunteer Preferences - Schedule.csv", usecols=top_artist_cols)

col_dict = {
        'Volunteer Top Artists': 'timeslot'
    , 'Unnamed: 20': 'Thursday'
    , 'Unnamed: 21': 'Friday'
    , 'Unnamed: 22': 'Saturday'
    , 'Unnamed: 23': 'Sunday'
}

preferences_matrix = preferences_matrix.rename(columns=col_dict).reset_index(drop=True)
preferences_matrix.drop([0, 1], inplace=True)
preferences_matrix.reset_index(drop=True, inplace=True)
preferences_matrix.fillna('', inplace=True)

In [55]:
#This one works
#preferences_matrix.iloc[preferences_matrix['timeslot']=='0030', preferences_matrix.columns.get_loc('Thursday')] = 'Max 1'

def write_act_to_matrix(day, start_time, end_time, counter, volunteer):
    #Timeslots that match for this act
    start_time = str(start_time.values[0]) 
    end_time = str(end_time.values[0]) 
    
    start = datetime.strptime(start_time, "%H%M") 
    end = datetime.strptime(end_time, "%H%M") 
      
    difference = end - start 
    seconds = difference.total_seconds() 
    minutes = seconds / 60
    
    # start index of time block
    start_index = preferences_matrix.loc[preferences_matrix['timeslot'] == start_time].index
    # number of slots occupied
    num_indices = int(minutes / 30)
    
    # end index of time block
    end_index = start_index + num_indices
    print(counter)
    # for each index, write it
    for i in range(0, num_indices):
        print(counter)
        str_value = vol_name + ' ' + str(counter) + ' '
        preferences_matrix.iloc[[start_index.values[0]+i], preferences_matrix.columns.get_loc(act_day.values[0])] += str_value
        
        #preferences_matrix.iloc[[start_index.values[0]+i], preferences_matrix.columns.get_loc(act_day.values[0])] = vol_name + str(i)
    #preferences_matrix.iloc[preferences_matrix['timeslot']=='0030', preferences_matrix.columns.get_loc(act_day)] = 'Max ' + str(act_num)

In [56]:
for index, row in step2_responses.iterrows():
    row_artists = []
    row_artists.append(row['Top Artist #1'])
    row_artists.append(row['Top Artist #2'])
    row_artists.append(row['Top Artist #3'])
    act_num = 0
    vol_name = row['fullName']
    
    for act in row_artists:
        act_num += 1
        
        try:
            new_act_name = artists.loc[artists['Original'] == act]['Renamed'].reset_index(drop=True)[0]
        except:
            new_act_name = 'T pain'
        act_day = set_times.loc[set_times['LookupName'] == new_act_name]['Day']
        act_start_time = set_times.loc[set_times['LookupName'] == new_act_name]['Start Time']
        act_end_time = set_times.loc[set_times['LookupName'] == new_act_name]['End Time']
        try:
            write_act_to_matrix(act_day, act_start_time, act_end_time, act_num, vol_name)
        except:
            pass

        

        

1
2
3
3
3
3
1
1
1
2
2
2
3
1
1
2
2
2
2
3
3
3
3
1
1
1
2
2
2
2
3
1
1
1
1
1
1
1
1
2
1
1
1
1
2
3
3
3
3
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
1
2
2
2
3
3
3
3
1
1
1
1
2
2
2
2
3
3
3
3
1
1
1
1
2
2
2
3
3
3
1
2
2
2
2
3
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
1
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
1
1
1
1
2
2
2
3
3
3
3
1
1
1
1
2
2
2
2
2
3
3
3
2
3
3
3
3
3
1
1
1
1
2
2
2
2
3
3
3
1
1
1
1
2
2
2
3
1
1
1
1
2
2
2
2
3
3
3
3
1
1
1
1
2
2
2
2
3
3
3
3
3
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
1
2
2
2
2
3
3
3
2
2
2
3
3
3
3
1
2
3
3
3
3
1
1
1
1
2
2
2
2
3
3
3
1
1
1
1
2
2
2
2
3
3
3
1
2
2
2
2
2
3
3
3
3
1
1
1
1
2
3
1
1
1
1
2
2
2
3
1
2
2
2
2
2
3
1
1
1
1
2
2
2
2
3
3
3
3
1
1
1
1
2
3
1
1
1
2
2
2
2
3
3
3
3
1
1
1
2
2
2
2
3
3
3
3


In [57]:
step2_responses

,fullName,matches,First name,Last name,Preferred Volunteer Partner,Top Artist #1,Top Artist #2,Top Artist #3
0,MauraPoole,Maura (Mo)Poole,Maura,Poole,Darby :),Red Hot Chili Peps 🔥 🌶️,Posty!,Cage the Elephant
1,DarbyHolmes,DarbyHolmes,Darby,Holmes,MO BABY!!!,Chappell roan,Brittany howard,Pretty lights sunrise
2,MonicaMendoza,MonicaMendoza,Monica,Mendoza,"Phil Suozzi, Lexton Deleon, and CJ Bradley!",Ludacris,Fisher,Ashnikko
3,PhilSuozzi,PhilSuozzi,Phil,Suozzi,Monica Mendoza,Sean Paul,Four tet,Pretty lights
4,Devin aka “Polaroid”Patrick,"Devin ""Polaroid""Patrick",Devin aka “Polaroid”,Patrick,I’m flexible,Khruangbin,Don’t really care about the music so much. I ...,Happy to be of service whenever needed
5,GwenMorbeck,GwenMorbeck,Gwen,Morbeck,Devin Patrick aka “Polaroid”,Khruangbin,Post Malone,I don’t particularly care about the music port...
6,Christian “Kip”Hunsinger,Christian (Kip)Hunsinger,Christian “Kip”,Hunsinger,Noah Keim,Cage the elephant,Pretty lights,GroupLove
7,AllisonMcKinley,AllisonMcKinley,Allison,McKinley,Ethan McKinley and Darren Van Paris,Red Hot Chili Peppers,T pain,Maggie Rogers
8,DarrenVan Paris,DarrenVan Paris,Darren,Van Paris,Allison & Ethan McKinley,Pretty Lights,Sean Paul,Fred Again
9,JessKessie,JessKessie,Jess,Kessie,Malia Rogers!!!,Fred again,Khruangbin,Rene rapp


In [58]:
preferences_matrix

,timeslot,Thursday,Friday,Saturday,Sunday
0,0000,,,,
1,0030,,MonicaMendoza 2,AllisonMcKinley 2 KieraMoore 2,NoahKeim 2 BriannaSpadafora 2
2,0100,,MonicaMendoza 2,AllisonMcKinley 2 KieraMoore 2 NicoMaciel 1,NoahKeim 2 JennaBerggren 3 NicoMaciel 3 Briann...
3,0130,,MonicaMendoza 2 ClaireNagy-Kato 3,AllisonMcKinley 2 KieraMoore 2 NicoMaciel 1,NoahKeim 2 JennaBerggren 3 NicoMaciel 3 Briann...
4,0200,,ClaireNagy-Kato 3,AllisonMcKinley 2 KieraMoore 2 NicoMaciel 1,NoahKeim 2 JennaBerggren 3 NicoMaciel 3 Briann...
5,0230,,,AllisonMcKinley 2 KieraMoore 2,NoahKeim 2 JennaBerggren 3 NicoMaciel 3 Briann...
6,0300,,,AllisonMcKinley 2 KieraMoore 2,NoahKeim 2 BriannaSpadafora 2
7,0330,,,AllisonMcKinley 2 KieraMoore 2,NoahKeim 2 BriannaSpadafora 2
8,0400,,,AllisonMcKinley 2 KieraMoore 2,NoahKeim 2 BriannaSpadafora 2
9,0430,,,AllisonMcKinley 2 KieraMoore 2,NoahKeim 2 BriannaSpadafora 2


In [59]:
preferences_matrix[['Thursday', 'Friday', 'Saturday', 'Sunday']]

,Thursday,Friday,Saturday,Sunday
0,,,,
1,,MonicaMendoza 2,AllisonMcKinley 2 KieraMoore 2,NoahKeim 2 BriannaSpadafora 2
2,,MonicaMendoza 2,AllisonMcKinley 2 KieraMoore 2 NicoMaciel 1,NoahKeim 2 JennaBerggren 3 NicoMaciel 3 Briann...
3,,MonicaMendoza 2 ClaireNagy-Kato 3,AllisonMcKinley 2 KieraMoore 2 NicoMaciel 1,NoahKeim 2 JennaBerggren 3 NicoMaciel 3 Briann...
4,,ClaireNagy-Kato 3,AllisonMcKinley 2 KieraMoore 2 NicoMaciel 1,NoahKeim 2 JennaBerggren 3 NicoMaciel 3 Briann...
5,,,AllisonMcKinley 2 KieraMoore 2,NoahKeim 2 JennaBerggren 3 NicoMaciel 3 Briann...
6,,,AllisonMcKinley 2 KieraMoore 2,NoahKeim 2 BriannaSpadafora 2
7,,,AllisonMcKinley 2 KieraMoore 2,NoahKeim 2 BriannaSpadafora 2
8,,,AllisonMcKinley 2 KieraMoore 2,NoahKeim 2 BriannaSpadafora 2
9,,,AllisonMcKinley 2 KieraMoore 2,NoahKeim 2 BriannaSpadafora 2


In [60]:
write_act_to_matrix(act_day, act_start_time, act_end_time, act_num, vol_name)

3
3
3
3


In [ ]:
'''
Ideal 
18-24 hours
one full day off for people working run of show
spot check double booking of different shift types per time slot

'''